## Metaclass
- Cria classe de forma dinâmica
- Pode ser útil na criação de frameworks onde as classes precisam ter um comportamento específico

In [ ]:
## Ele herda de type que cria todas as classes
## O método __new__ é como se fosse o __init__ para classes
## O return ele retorna para o type terminar de criar a classe
class MeuMeta(type):
    def __new__(cls, nome, bases, dct):
        dct['novo_atributo'] = "Valor adicionado pela metaclasse"
        return super().__new__(cls, nome, bases, dct)

In [ ]:
## Necessário colocar 'metaclass=' para funcionar
class MinhaClasse(metaclass=MeuMeta):
    pass

In [5]:
obj = MinhaClasse()
obj.novo_atributo

'Valor adicionado pela metaclasse'

In [32]:
class ValidadorMeta(type):
    def __new__(cls, nome, bases, dct: dict):
        # Dicionário para armazenar as validações
        validacoes = dct.get("validacoes", {})
        print(f"Validações: {validacoes}")
        for attr, tipo in validacoes.items():
            ## Classes são callable, como também instâncias de classe com o método __call__()
            if not callable(tipo):
                raise ValueError(f"O tipo de validação para {attr} deve ser uma função.")
            
            # Adiciona uma nova função de validação
            # 'value' é o valor que é passado ao dar set_nome("ex") por exemplo, ele verifica se é uma string -> 'tipo'
                # Exemplo de como fica ao iniciar uma classe
                    # |  set_nome(self, value, attr='nome', tipo=<class 'str'>) from __main__.ValidadorMeta.__new__.<locals>
                    # |      # Adiciona uma nova função de validação
                    # |      # 'value' é o valor que é passado ao dar set_nome por exemplo, ele verifica se é uma string -> 'attr'
            # Método de verificação de tipos ao inicializar a variável
            def valida_func(self, value, attr=attr, tipo=tipo):
                # faz esta verificação neste if e gera o erro, retornando que o 
                if not isinstance(value, tipo):
                    raise ValueError(f"'{attr}' deve ser do tipo {tipo.__name__}")
                
                self.__dict__[attr] = value

            # Renomeia a função para evitar problemas de escopo
            valida_func.__name__ = f"set_{attr}"

            # Atribui a função de validação criada acima
            dct[f"set_{attr}"] = valida_func

        return super().__new__(cls, nome, bases, dct)

teste = 1
class Usuario(metaclass=ValidadorMeta):
    validacoes = {
        "nome": str,
        "idade": int,
        ## Gera o primeiro erro, pois não é uma função, exemplos : int, float, bool, str são uma classe
        # "teste": teste
    }

    def __init__(self, nome, idade):
        self.set_nome(nome)
        self.set_idade(idade)


Validações: {'nome': <class 'str'>, 'idade': <class 'int'>}


In [29]:
user = Usuario("Alice", 30)
print(f"Nome: {user.nome}, Idade: {user.idade}")
help(user)


# user.set_idade("trinta")

user.idade = "teste"
print(f"teste idade: {user.idade}")

Nome: Alice, Idade: 30
Help on Usuario in module __main__ object:

class Usuario(builtins.object)
 |  Usuario(nome, idade)
 |
 |  Methods defined here:
 |
 |  __init__(self, nome, idade)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  set_idade(self, value, attr='idade', tipo=<class 'int'>) from __main__.ValidadorMeta.__new__.<locals>
 |      # Método de verificação de tipos ao inicializar a variável
 |
 |  set_nome(self, value, attr='nome', tipo=<class 'str'>) from __main__.ValidadorMeta.__new__.<locals>
 |      # Método de verificação de tipos ao inicializar a variável
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  validacoes = {'idade': <cla